In [93]:
import os
import os.path
import pandas as pd
import pickle
from PIL import Image, ImageDraw
import re

In [94]:
def readPickle(filename):

    # reload a file to a variable
    with open(filename, 'rb') as file:
        a_dict1 =pickle.load(file)
        
    return a_dict1

In [95]:
def drawPicture(name, xfFix, yfFix, xfTracing, yfTracing, folder):
    
    name = nameJude(name, folder)
    image = Image.open(r'..//img/back.png')
    
    draw = ImageDraw.Draw(image)

    drawObject(draw, xfFix, yfFix, 255, 0, 0, 255, 0, 255)
    drawObject(draw, xfTracing, yfTracing, 0, 0, 255, 0, 255, 255)
        
    image.save(r'..//' + folder + '/' + name + '.png')

In [96]:
def drawObject(draw, xf, yf, eR, eG, eB, lR, lG, lB):
    
    bxf = 0 
    byf = 0

    #画像に線を引く
    for i in range(len(xf)):
        for j in range(1, len(xf[i])):
            if bxf is not 0: 
                draw.line((bxf, byf, xf[i][0], yf[i][0]), fill=(lR, lG, lB), width=10)
                bxf = 0
                byf = 0
                
            draw.line((xf[i][j-1], yf[i][j-1], xf[i][j], yf[i][j]), fill=(lR, lG, lB), width=10)
            
            if j is len(xf[i])-1: 
                bxf=xf[i][j]
                byf=yf[i][j]
            
        if len(xf[i]) is not 0: draw.ellipse((xf[i][0]-10, yf[i][0]-10, xf[i][0]+10, yf[i][0]+10), 
                                                                      fill=(eR, eG, eB), outline=None)

SyntaxError: invalid syntax (<ipython-input-96-6ae59571c6d1>, line 11)

In [91]:
#ファイル名を検索して、決定するメソッド
def nameJude(name, folder):
    
    #保存フォルダが持つファイルの取得
    files = os.listdir(r'..//' + folder)
    
    #保存できない記号の変更
    name = name.replace('<', '(back)')
    name = name + '_'
    
    count = 0
    for file in files:
        index = re.search(name, file)
        if index: count+=1
        
    name = name + str(count+1)
    
    return name

In [92]:
dictFix = readPickle('fixationTotalDict.pkl')['506']['1']
dictTracing = readPickle('tracingTotalDict.pkl')['506']['1']
folder = 'result'

#このデータが所持しているsentenceを出す
sentence = []
for index in dictFix.keys():
    sentence.append(index)
    
##################################

#sentenceの数だけ回す    
for index in range(len(sentence)):
    
##########
    
    spacePlace = [] #スペースの位置を保管
    
    for space in range(len(dictFix[sentence[index]]['key'])):
        if dictFix[sentence[index]]['key'][space] is ' ' : spacePlace.append(space)
   
    #####
    
    
    #値の初期化
    start = 1 #wordの始まり
    
    #空白までの情報を取る
    for space in range(len(spacePlace)): 
        xfFix = [] #Fixaction xf
        yfFix = [] #Fixaction yf
        xfTracing = [] #Tracing xf
        yfTracing = [] #Tracing yf
    
        name = dictFix[sentence[index]]['key'][start-1] #ファイル名
        for pick in range(start, spacePlace[space]):
            name = name + dictFix[sentence[index]]['key'][pick]
            xfFix.append(dictFix[sentence[index]]['xf'][pick])
            yfFix.append(dictFix[sentence[index]]['yf'][pick])
            xfTracing.append(dictTracing[sentence[index]]['xf'][pick])
            yfTracing.append(dictTracing[sentence[index]]['yf'][pick])
            
        #####
        
        
        
        #画像に描写する
        drawPicture(name, xfFix, yfFix, xfTracing, yfTracing, folder)
                
        #####
        
        start = spacePlace[space]+2